# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [63]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

633

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [64]:
# Save api_key information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}"
print(query_url) 


http://api.openweathermap.org/data/2.5/weather?appid=16511068a487c94d8da78707ea12cd98&units=imperial


In [65]:
#Create lists for dataframe and API Calls 
city_name = []
country = []
clouds = []
lat = []
lngs = []
temp = []
max_temp = []
min_temp = []
humidity = []
wind = []

#Keep count
city_record = 1

#Statement of API log 
print("Data Retreieved:")
print("---------------------")

#Loop through cities in lists:
for city in cities:
    try:
        response = requests.get(f"{query_url}&q={city}").json()
        city_name.append(response["name"])
        country.append(response["sys"]["country"])
        clouds.append(response['clouds']['all'])
        lat.append(response["coord"]["lat"])
        lngs.append(response["coord"]["lon"])
        temp.append(response["main"]["temp"])
        max_temp.append(response["main"]["temp_max"])
        min_temp.append(response["main"]["temp_min"])
        humidity.append(response["main"]["humidity"])
        wind.append(response["wind"]["speed"])

        print(f"Displaying Record #{city_record} for {city}")

        #Increase counter
        city_record += 1

    #If no city found, handle for exception:
    except Exception:
        print("City not found, next city...")
        pass

Data Retreieved:
---------------------
Displaying Record #1 for bluff
Displaying Record #2 for ushuaia
Displaying Record #3 for bredasdorp
Displaying Record #4 for torbay
Displaying Record #5 for tuktoyaktuk
Displaying Record #6 for jamestown
Displaying Record #7 for trinidad
Displaying Record #8 for dalvik
Displaying Record #9 for surt
Displaying Record #10 for atuona
Displaying Record #11 for rikitea
Displaying Record #12 for new norfolk
Displaying Record #13 for kodiak
Displaying Record #14 for kruisfontein
Displaying Record #15 for thompson
Displaying Record #16 for hermanus
Displaying Record #17 for bambous virieux
City not found, next city...
Displaying Record #18 for santa rosa
Displaying Record #19 for tuatapere
Displaying Record #20 for hobyo
Displaying Record #21 for hun
City not found, next city...
Displaying Record #22 for bontoc
Displaying Record #23 for kabo
Displaying Record #24 for georgetown
Displaying Record #25 for albany
Displaying Record #26 for busselton
Displayin

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [91]:
#Convert data to DataFrame:
weather_data = {
    "City": city_name,
    "Country": country,
    "Latitude": lat,
    "Longitude": lngs,
    "Cloudiness": clouds,
    "Temperature": temp,
    "Max Temp": max_temp,
    "Min Temp": min_temp,
    "Humidity": humidity,
    "Wind Speed": wind
}

city_weather_df = pd.DataFrame(weather_data)
city_weather_df.head()



,City,Country,Latitude,Longitude,Cloudiness,Temperature,Max Temp,Min Temp,Humidity,Wind Speed
0,Bluff,NZ,-46.6000,168.3333,100,52.81,52.81,52.81,58,2.35
1,Ushuaia,AR,-54.8000,-68.3000,20,42.46,42.46,42.46,49,17.27
2,Bredasdorp,ZA,-34.5322,20.0403,12,57.02,57.02,57.02,67,18.19
3,Torbay,CA,47.6666,-52.7314,75,67.19,68.58,65.16,92,23.02
4,Tuktoyaktuk,CA,69.4541,-133.0374,100,55.40,55.40,55.40,94,5.75


In [72]:
#Export Dataframe to CSV
city_weather_df.to_csv("weather_data.csv", index=False, header=True)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [92]:
#Check if Humidity values over 100
city_weather_df["Humidity"].describe()


count    585.000000
mean      71.215385
std       20.330104
min        1.000000
25%       61.000000
50%       75.000000
75%       87.000000
max      100.000000
Name: Humidity, dtype: float64

In [96]:
#  Get the indices of cities that have humidity over 100%.
humidity_over = city_weather_df[(city_weather_df["Humidity"] >= 100)].index
humidity_over


Int64Index([], dtype='int64')

In [94]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

clean_city_data = city_weather_df.drop(humidity_over, inplace=False)
clean_city_data

,City,Country,Latitude,Longitude,Cloudiness,Temperature,Max Temp,Min Temp,Humidity,Wind Speed
0,Bluff,NZ,-46.6000,168.3333,100,52.81,52.81,52.81,58,2.35
1,Ushuaia,AR,-54.8000,-68.3000,20,42.46,42.46,42.46,49,17.27
2,Bredasdorp,ZA,-34.5322,20.0403,12,57.02,57.02,57.02,67,18.19
3,Torbay,CA,47.6666,-52.7314,75,67.19,68.58,65.16,92,23.02
4,Tuktoyaktuk,CA,69.4541,-133.0374,100,55.40,55.40,55.40,94,5.75
...,...,...,...,...,...,...,...,...,...,...
580,Sabzawār,IR,36.2126,57.6819,0,58.28,58.28,58.28,25,9.22
581,Acarí,PE,-15.4311,-74.6158,45,63.27,63.27,63.27,70,1.95
582,Tarauacá,BR,-8.1614,-70.7656,2,71.71,71.71,71.71,57,0.74
583,Illela,NG,13.7292,5.2975,100,78.04,78.04,78.04,78,6.20


In [95]:
#Export clean data to csv
clean_city_data.to_csv("clean_city_data.csv", index=False, header=True)

In [97]:
clean_city_data["Humidity"].describe()


count    585.000000
mean      71.215385
std       20.330104
min        1.000000
25%       61.000000
50%       75.000000
75%       87.000000
max      100.000000
Name: Humidity, dtype: float64

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression